In [1]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [4]:
data=pd.read_csv("most_frequent_words_30.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
835,anime,help,95
836,anime,begin,94
837,anime,father,93
838,anime,attack,91


In [5]:
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
dict_words


{'western': ['town',
  'men',
  'take',
  'back',
  'one',
  'kill',
  'man',
  'ranch',
  'horse',
  'find',
  'ride',
  'killed',
  'go',
  'gang',
  'return',
  'two',
  'get',
  'gun',
  'sheriff',
  'tell',
  'indian',
  'shoot',
  'father',
  'brother',
  'help',
  'try',
  'woman',
  'gold',
  'new',
  'come'],
 'comedy': ['get',
  'find',
  'go',
  'one',
  'take',
  'tell',
  'friend',
  'back',
  'love',
  'new',
  'two',
  'father',
  'make',
  'life',
  'day',
  'home',
  'time',
  'family',
  'house',
  'also',
  'try',
  'come',
  'meet',
  'end',
  'see',
  'later',
  'man',
  'film',
  'money',
  'however'],
 'short': ['tom',
  'jerry',
  'bug',
  'stooge',
  'get',
  'back',
  'cat',
  'head',
  'spike',
  'curly',
  'mouse',
  'run',
  'go',
  'try',
  'one',
  'house',
  'end',
  'moe',
  'see',
  'away',
  'take',
  'make',
  'door',
  'two',
  'dog',
  'time',
  'chase',
  'fall',
  'sam',
  'find'],
 'action': ['kill',
  'one',
  'police',
  'get',
  'find',
  'ta

In [6]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [7]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]
Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)

Wall time: 1min 9s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize

In [9]:
Train

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Plot_trimmed
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,film open two bandit breaking railroad telegra...,two two kill kill one try horse back tell men ...
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,film family move suburb hoping quiet life thin...,film family life go get
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...,rarebit fiend gorge welsh rarebit restaurant l...,get try get fall head back get fall
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,action,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,two later
12,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,two horse
...,...,...,...,...,...,...,...,...,...,...,...
42526,2013,Particle (film),Turkish,Erdem Tepegöz,"Jale Arıkan, Rüçhan Caliskur, Özay Fecht, Remz...",drama film,drama,https://en.wikipedia.org/wiki/Particle_(film),"Zeynep lost her job at weaving factory, and he...",zeynep lost job weaving factory family mother ...,family mother find new
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...,two
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...,father also friend try family find
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...,film however two go man go man new life get life


In [21]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 5000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(Train['Plot_trimmed'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 178 unique tokens.


In [22]:
X = tokenizer.texts_to_sequences(Train['Plot_trimmed'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (35692, 300)


In [23]:
X_test1 = tokenizer.texts_to_sequences(Test['Plot_cleanned'].values)
X_test1 = pad_sequences(X_test1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test1.shape)

Shape of data tensor: (6843, 300)


In [24]:
Y = pd.get_dummies(Train['Genre_updated']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (35692, 28)


In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(24984, 300) (24984, 28)
(10708, 300) (10708, 28)


In [26]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(28, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          500000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 300, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 28)                2828      
Total params: 583,228
Trainable params: 583,228
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
352/352 [==============================] - 402s 1s/step - loss: 1.8162 - accuracy: 0.4706 - val_loss: 1.1039 - val_accuracy: 0.6715
Epoch 2/5
352/352 [==============================] - 410s 1s/step - loss: 0.9968 - accuracy: 0.6993 - val_loss: 0.8410 - val_accuracy: 0.7543
Epoch 3/5
352/352 [==============================] - 424s 1s/step - loss: 0.7982 - accuracy: 0.7544 - val_loss: 0.7042 - val_accuracy: 0.7863
Epoch 4/5
352/352 [==============================] - 419s 1s/step - loss: 0.7001 - accuracy: 0.7763 - val_loss: 0.6521 - val_accuracy: 0.7835
Epoch 5/5
352/352 [==============================] - 407s 1s/step - loss: 0.6395 - accuracy: 0.7894 - val_loss: 0.6132 - val_accuracy: 0.8023


In [28]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

335/335 [==============================] - 18s 53ms/step - loss: 0.6164 - accuracy: 0.7920
Test set
  Loss: 0.616
  Accuracy: 0.792


In [29]:
y_pred=model.predict(X_test)

In [30]:
model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([ 2,  1,  8, ...,  5, 17, 13], dtype=int64)

In [31]:
dx=pd.DataFrame(model.predict(X_test1),columns=['action', 'adventure', 'animated', 'anime', 'biography', 'comedy',
       'crime', 'documentary', 'drama', 'family', 'fantasy', 'historical',
       'horror', 'martial_arts', 'musical', 'mystery', 'noir', 'romance',
       'sci_fi', 'serial', 'short', 'social', 'sports', 'spy', 'teen',
       'thriller', 'war', 'western'])

In [32]:
dx

,action,adventure,animated,anime,biography,comedy,crime,documentary,drama,family,...,sci_fi,serial,short,social,sports,spy,teen,thriller,war,western
0,0.000004,0.002063,0.001527,0.002587,0.000883,0.000002,0.000006,0.002937,0.000004,0.000009,...,0.002270,0.205873,0.012616,0.000004,0.000108,0.034735,0.000052,0.000008,0.732639,6.125135e-06
1,0.000045,0.001025,0.007395,0.000178,0.000491,0.000018,0.004610,0.000096,0.000085,0.000687,...,0.000747,0.016115,0.013865,0.012925,0.007547,0.005034,0.001092,0.001398,0.000139,1.094349e-01
2,0.000073,0.000956,0.024305,0.009684,0.161989,0.000249,0.000018,0.329348,0.000082,0.002517,...,0.000370,0.019487,0.173794,0.000881,0.007085,0.015995,0.014956,0.000060,0.206480,5.034728e-05
3,0.000028,0.000126,0.001842,0.004261,0.001992,0.000005,0.000060,0.010026,0.000012,0.000018,...,0.001227,0.400873,0.010270,0.000162,0.027190,0.391510,0.001641,0.000016,0.095974,1.298300e-03
4,0.000133,0.000468,0.007444,0.011231,0.166389,0.000011,0.000038,0.084909,0.000045,0.001213,...,0.000279,0.041162,0.010713,0.001913,0.102771,0.067669,0.122220,0.000082,0.007648,1.072715e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6838,0.000844,0.001436,0.000548,0.006752,0.240136,0.000013,0.000003,0.037678,0.010169,0.127185,...,0.000960,0.000036,0.000056,0.088000,0.000620,0.000119,0.004079,0.000429,0.000676,1.119073e-05
6839,0.000084,0.001992,0.547107,0.003051,0.000499,0.000139,0.000968,0.003197,0.000007,0.003882,...,0.000245,0.037772,0.102628,0.003190,0.002366,0.086604,0.045816,0.000086,0.007829,3.787869e-03
6840,0.000163,0.000653,0.166121,0.049546,0.005861,0.000053,0.000046,0.019220,0.000023,0.011912,...,0.000229,0.009234,0.004025,0.014638,0.001141,0.049404,0.049099,0.000222,0.045829,2.500644e-04
6841,0.000171,0.000214,0.178135,0.014400,0.004748,0.000030,0.000049,0.008093,0.000006,0.002765,...,0.000098,0.027164,0.101849,0.006907,0.054592,0.326255,0.080295,0.000076,0.008228,3.010949e-03


In [33]:
def convert(x):
    if x>0.1:
        return 1
    return 0
    
for i in dx.columns:
    dx[i]=dx[i].apply(lambda x:convert(x))

In [34]:
dx.sum(axis=1).value_counts()

2    2476
3    1872
1    1632
4     726
5     131
6       5
0       1
dtype: int64

In [35]:
def genre_pred(col,x):

    if x==1:
        return col

for i in dx.columns:

    dx[i]=dx[i].apply(lambda x:genre_pred(i,x))

In [36]:
dx_list=dx.to_numpy().tolist()

In [37]:
Test['Genre_pred']=dx_list

<ipython-input-37-1e867a2d28ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Genre_pred']=dx_list


In [38]:
Test['Genre_pred']=Test['Genre_pred'].apply(lambda x:[i for i in x if i])

<ipython-input-38-5d7dce274ac8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test['Genre_pred']=Test['Genre_pred'].apply(lambda x:[i for i in x if i])


In [39]:
Test

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Genre_pred
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...,"[serial, war]"
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...,"[mystery, noir, western]"
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...,"[biography, documentary, short, war]"
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...,"[serial, spy]"
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...,"[biography, historical, sports, teen]"
...,...,...,...,...,...,...,...,...,...,...,...
42482,2009,Orada,Turkish,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Orada,When the matriarch of an Istanbul family dies ...,matriarch istanbul family dy home elderly son ...,"[biography, family, historical]"
42527,2014,Mandıra Filozofu,Turkish,Director: Müfit Can Saçıntı,Director: Müfit Can Saçıntı\r\nCast: Rasim Özt...,unknown,NaN,https://en.wikipedia.org/wiki/Mand%C4%B1ra_Fil...,Cavit an ambitious industralist in İstanbul pl...,cavit ambitious industralist i̇stanbul plan bu...,"[animated, short]"
42528,2014,Winter Sleep,Turkish,Director: Nuri Bilge Ceylan,Director: Nuri Bilge Ceylan\r\nCast: Haluk Bil...,unknown,NaN,https://en.wikipedia.org/wiki/Winter_Sleep_(film),"Aydın, a former actor, owns a mountaintop hote...",aydın former actor owns mountaintop hotel capp...,"[animated, historical]"
42529,2014,Sivas,Turkish,Director: Kaan Müjdeci,Director: Kaan Müjdeci\r\nCast: Dogan Izci,unknown,NaN,https://en.wikipedia.org/wiki/Sivas_(film),The film follows an eleven-year-old boy named ...,film follows elevenyearold boy named aslan dog...,"[animated, short, spy]"
